In [1]:
!pip install pymysql sqlalchemy

In [2]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

import getpass  # to get the password without showing the input
password = getpass.getpass()

connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)

 ········


In [3]:
# Getting the data.
query = '''SELECT c.customer_id, c.first_name, c.last_name, COUNT(r.rental_id) AS rental_count
           FROM rentals_may r
           JOIN customer c ON r.customer_id = c.customer_id
           GROUP BY c.customer_id, c.last_name;'''

data = pd.read_sql_query(query, engine)
data

,customer_id,first_name,last_name,rental_count
0,130,CHARLOTTE,HUNTER,4
1,459,TOMMY,COLLAZO,2
2,408,MANUEL,MURRELL,6
3,333,ANDREW,PURDY,2
4,222,DELORES,HANSEN,10
...,...,...,...,...
515,191,JEANETTE,GREENE,4
516,351,JACK,FOUST,2
517,10,DOROTHY,TAYLOR,2
518,136,ANITA,MORALES,2


In [7]:
# Check the number of rentals for each customer for May.
rentals_may = """
SELECT c.customer_id, c.first_name, c.last_name, COUNT(r.rental_id) AS rental_count
FROM rentals_may r
JOIN customer c ON r.customer_id = c.customer_id
GROUP BY c.customer_id, c.last_name;
"""

In [8]:
# Check the number of rentals for each customer for June.
rentals_june = """
SELECT c.customer_id, c.first_name, c.last_name, COUNT(r.rental_id) AS rental_count
FROM rentals_june r
JOIN customer c ON r.customer_id = c.customer_id
GROUP BY c.customer_id, c.last_name;
"""

In [10]:
df_may = pd.read_sql(rentals_may, engine)
df_may

,customer_id,first_name,last_name,rental_count
0,130,CHARLOTTE,HUNTER,4
1,459,TOMMY,COLLAZO,2
2,408,MANUEL,MURRELL,6
3,333,ANDREW,PURDY,2
4,222,DELORES,HANSEN,10
...,...,...,...,...
515,191,JEANETTE,GREENE,4
516,351,JACK,FOUST,2
517,10,DOROTHY,TAYLOR,2
518,136,ANITA,MORALES,2


In [11]:
df_june = pd.read_sql(rentals_june, engine)
df_june

,customer_id,first_name,last_name,rental_count
0,416,JEFFERY,PINSON,40
1,516,ELMER,NOE,48
2,239,MINNIE,ROMERO,40
3,285,MIRIAM,MCKINNEY,24
4,310,DANIEL,CABRAL,48
...,...,...,...,...
585,412,ALLEN,BUTTERFIELD,8
586,335,GREGORY,MAULDIN,8
587,226,MAUREEN,LITTLE,16
588,22,LAURA,RODRIGUEZ,8


In [15]:
# Check if customer borrowed more or less films in the month of June as compared to May.
df_comparison = pd.merge(df_may, df_june, on='customer_id', how='outer', suffixes=('_may', '_june'))

# Fill missing values with 0 (for customers who borrowed in one month but not the other).
df_comparison.fillna(0, inplace=True)

# Add a new column to compare rentals in June vs May.
def compare(row):
        if row['rental_count_june'] > row['rental_count_may']:
            return 'More in June'
        elif row['rental_count_june'] < row['rental_count_may']:
            return 'Less in June'
        else:
            return 'Same in both months'

df_comparison['comparison'] = df_comparison.apply(compare, axis=1)
df_comparison

,customer_id,first_name_may,last_name_may,rental_count_may,first_name_june,last_name_june,rental_count_june,comparison
0,130,CHARLOTTE,HUNTER,4.0,CHARLOTTE,HUNTER,56.0,More in June
1,459,TOMMY,COLLAZO,2.0,TOMMY,COLLAZO,56.0,More in June
2,408,MANUEL,MURRELL,6.0,MANUEL,MURRELL,24.0,More in June
3,333,ANDREW,PURDY,2.0,ANDREW,PURDY,32.0,More in June
4,222,DELORES,HANSEN,10.0,DELORES,HANSEN,16.0,More in June
...,...,...,...,...,...,...,...,...
593,598,0,0,0.0,WADE,DELVALLE,8.0,More in June
594,487,0,0,0.0,HECTOR,POINDEXTER,8.0,More in June
595,555,0,0,0.0,DWIGHT,LOMBARDI,8.0,More in June
596,335,0,0,0.0,GREGORY,MAULDIN,8.0,More in June
